In [1]:
from pybaseball import statcast

In [2]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [3]:
#hitter_stats = statcast(start_dt='2023-3-30', end_dt='2023-6-30')

In [4]:
#hitter_stats.to_csv('statcast.csv',index=False)

In [5]:
statcast_class = pd.read_csv("C:/Users/wei/Documents/實習/球探報告/statcast_XGB.csv")
statcast = pd.read_csv("C:/Users/wei/Documents/實習/球探報告/statcast.csv")

In [6]:
baseball_cols = ['game_year','game_date','player_name','des', 'pitcher','batter', 'pitch_type', 'pitch_name', 'events','description','bb_type'
                ,'zone','stand'] 

pitch_stat = statcast[baseball_cols]

In [7]:
class_col = ['name','class_pred']
statcast_class = statcast_class[class_col]

In [8]:
statcast_class

,name,class_pred
0,Trea Turner,B
1,Adley Rutschman,D
2,Justin Turner,D
3,Jonathan India,B
4,Bobby Dalbec,B
...,...,...
525,Nolan Arenado,A
526,Christian Walker,A
527,Nathaniel Lowe,A
528,Andrew Benintendi,A


In [9]:
statcast_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        530 non-null    object
 1   class_pred  530 non-null    object
dtypes: object(2)
memory usage: 8.4+ KB


##  打者名字處理

In [10]:
name_parts = pitch_stat.des.str.split(" ")

In [11]:
name_parts.head(3)

0    [Brandon, Marsh, strikes, out, swinging.]
1    [Brandon, Marsh, strikes, out, swinging.]
2    [Brandon, Marsh, strikes, out, swinging.]
Name: des, dtype: object

In [12]:
batter_name = name_parts.apply(lambda x: x[:2])

In [13]:
batter_name.head(3)

0    [Brandon, Marsh]
1    [Brandon, Marsh]
2    [Brandon, Marsh]
Name: des, dtype: object

In [14]:
statcast = pitch_stat.assign(batter_name=batter_name)

In [15]:
statcast['batter_name'] = statcast['batter_name'].str[0].str.replace('[', '').str.replace(']', '') +' '+ statcast['batter_name'].str[1].str.replace('[', '').str.replace(']', '')

In [16]:
statcast['batter_name']

0         Brandon Marsh
1         Brandon Marsh
2         Brandon Marsh
3             Alec Bohm
4             Alec Bohm
              ...      
363032      Steven Kwan
363033      Steven Kwan
363034      Steven Kwan
363035      Steven Kwan
363036      Steven Kwan
Name: batter_name, Length: 363037, dtype: object

## 整理完後的資料(+打者名字)

In [17]:
statcast

,game_year,game_date,player_name,des,pitcher,batter,pitch_type,pitch_name,events,description,bb_type,zone,stand,batter_name
0,2023,2023-06-30,"Harvey, Hunter",Brandon Marsh strikes out swinging.,640451,669016,FS,Split-Finger,strikeout,swinging_strike_blocked,NaN,13.0,L,Brandon Marsh
1,2023,2023-06-30,"Harvey, Hunter",Brandon Marsh strikes out swinging.,640451,669016,FF,4-Seam Fastball,NaN,called_strike,NaN,9.0,L,Brandon Marsh
2,2023,2023-06-30,"Harvey, Hunter",Brandon Marsh strikes out swinging.,640451,669016,FF,4-Seam Fastball,NaN,called_strike,NaN,11.0,L,Brandon Marsh
3,2023,2023-06-30,"Harvey, Hunter","Alec Bohm grounds out, third baseman Jeimer Ca...",640451,664761,SL,Slider,field_out,hit_into_play,ground_ball,14.0,R,Alec Bohm
4,2023,2023-06-30,"Harvey, Hunter","Alec Bohm grounds out, third baseman Jeimer Ca...",640451,664761,SL,Slider,NaN,called_strike,NaN,8.0,R,Alec Bohm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363032,2023,2023-03-30,"Castillo, Luis",Steven Kwan pops out to third baseman Eugenio ...,622491,680757,SI,Sinker,NaN,called_strike,NaN,11.0,L,Steven Kwan
363033,2023,2023-03-30,"Castillo, Luis",Steven Kwan pops out to third baseman Eugenio ...,622491,680757,FF,4-Seam Fastball,NaN,called_strike,NaN,1.0,L,Steven Kwan
363034,2023,2023-03-30,"Castillo, Luis",Steven Kwan pops out to third baseman Eugenio ...,622491,680757,FF,4-Seam Fastball,NaN,ball,NaN,12.0,L,Steven Kwan
363035,2023,2023-03-30,"Castillo, Luis",Steven Kwan pops out to third baseman Eugenio ...,622491,680757,SI,Sinker,NaN,ball,NaN,13.0,L,Steven Kwan


In [18]:
statcast.pitch_name.value_counts(dropna=False)

4-Seam Fastball    117052
Slider              65247
Sinker              54714
Changeup            39146
Cutter              28239
Curveball           25994
Sweeper             15269
Split-Finger         7957
Knuckle Curve        6742
Slurve               1246
Other                 574
Forkball              419
Eephus                243
NaN                    76
Screwball              52
Slow Curve             29
Pitch Out              20
Knuckleball            18
Name: pitch_name, dtype: int64

In [19]:
# 將"game_date"欄位轉換為日期格式
statcast['game_date'] = pd.to_datetime(statcast['game_date'])

## 篩選球員 球種

In [53]:
# 輸入打者名稱和pitch_type
batter_name ="Juan Soto"
pitch_type = 'FF'

In [54]:
#投手 姓氏需倒過來 + "," + 空格
#EX:Logan Webb --> Webb, Logan 
pitcher_name ="Cole, Gerrit"

In [55]:
batter_data = statcast[(statcast['batter_name'] == batter_name) & (statcast['pitch_type'] == pitch_type)& (statcast['game_date'] < '2023-06-28')]

In [56]:
batter_data.zone.value_counts(dropna=False)

11.0    72
12.0    54
2.0     41
13.0    33
1.0     29
4.0     26
14.0    23
5.0     19
8.0     19
3.0     18
6.0     18
7.0     16
9.0     12
Name: zone, dtype: int64

In [57]:
corner = []

# 使用迴圈遍歷資料並進行條件檢查
for index, row in batter_data.iterrows():
    if row['stand'] == 'R' and row['zone'] == 1:
        corner_ID = '內角高好球'
    elif row['stand'] == 'R' and row['zone'] == 2:
        corner_ID = '中間高好球'
    elif row['stand'] == 'R' and row['zone'] == 3:
        corner_ID = '外角高好球'
    elif row['stand'] == 'R' and row['zone'] == 4:
        corner_ID = '內角中好球'
    elif row['stand'] == 'R' and row['zone'] == 5:
        corner_ID = '紅中'
    elif row['stand'] == 'R' and row['zone'] == 6:
        corner_ID = '外角中好球'
    elif row['stand'] == 'R' and row['zone'] == 7:
        corner_ID = '內角低好球'
    elif row['stand'] == 'R' and row['zone'] == 8:
        corner_ID = '中間低好球'
    elif row['stand'] == 'R' and row['zone'] == 9:
        corner_ID = '外角低好球'
    elif row['stand'] == 'R' and row['zone'] == 11:
        corner_ID = '內角高壞球'
    elif row['stand'] == 'R' and row['zone'] == 12:
        corner_ID = '外角高壞球'
    elif row['stand'] == 'R' and row['zone'] == 13:
        corner_ID = '內角低壞球'
    elif row['stand'] == 'R' and row['zone'] == 14:
        corner_ID = '外角低壞球'
#---------------我是分隔線--------------------- 
    elif row['stand'] == 'L' and row['zone'] == 3:
        corner_ID = '內角高好球'       
    elif row['stand'] == 'L' and row['zone'] == 2:
        corner_ID = '中間高好球'
    elif row['stand'] == 'L' and row['zone'] == 1:
        corner_ID = '外角高好球'
    elif row['stand'] == 'L' and row['zone'] == 4:
        corner_ID = '內角中好球'
    elif row['stand'] == 'L' and row['zone'] == 5:
        corner_ID = '紅中'
    elif row['stand'] == 'L' and row['zone'] == 4:
        corner_ID = '外角中好球'
    elif row['stand'] == 'L' and row['zone'] == 9:
        corner_ID = '內角低好球'
    elif row['stand'] == 'L' and row['zone'] == 8:
        corner_ID = '中間低好球'
    elif row['stand'] == 'L' and row['zone'] == 7:
        corner_ID = '外角低好球'
    elif row['stand'] == 'L' and row['zone'] == 12:
        corner_ID = '內角高壞球'
    elif row['stand'] == 'L' and row['zone'] == 11:
        corner_ID = '外角高壞球'
    elif row['stand'] == 'L' and row['zone'] == 14:
        corner_ID = '內角低壞球'
    else:
        corner_ID = '外角低壞球'
    
    # 將結果新增至 "corner" 列表
    corner.append(corner_ID)

# 將 "corner" 列表新增為新的欄位 "corner" 至 statcast 資料
batter_data['corner'] = corner

In [59]:
batter_data

,game_year,game_date,player_name,des,pitcher,batter,pitch_type,pitch_name,events,description,bb_type,zone,stand,batter_name,corner
13006,2023,2023-06-27,"Contreras, Roansy",Juan Soto flies out to left fielder Josh Palac...,672710,665742,FF,4-Seam Fastball,field_out,hit_into_play,fly_ball,2.0,L,Juan Soto,中間高好球
13009,2023,2023-06-27,"Contreras, Roansy",Juan Soto flies out to left fielder Josh Palac...,672710,665742,FF,4-Seam Fastball,NaN,foul,NaN,1.0,L,Juan Soto,外角高好球
13156,2023,2023-06-27,"Hill, Rich",Juan Soto walks.,448179,665742,FF,4-Seam Fastball,NaN,foul,NaN,2.0,L,Juan Soto,中間高好球
13157,2023,2023-06-27,"Hill, Rich",Juan Soto walks.,448179,665742,FF,4-Seam Fastball,NaN,ball,NaN,13.0,L,Juan Soto,外角低壞球
13231,2023,2023-06-27,"Hill, Rich",Juan Soto called out on strikes.,448179,665742,FF,4-Seam Fastball,NaN,foul,NaN,11.0,L,Juan Soto,外角高壞球
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357373,2023,2023-03-31,"Hand, Brad",Juan Soto called out on strikes.,543272,665742,FF,4-Seam Fastball,NaN,called_strike,NaN,8.0,L,Juan Soto,中間低好球
357566,2023,2023-03-31,"Freeland, Kyle",Juan Soto walks.,607536,665742,FF,4-Seam Fastball,NaN,called_strike,NaN,2.0,L,Juan Soto,中間高好球
359911,2023,2023-03-30,"Márquez, Germán","Juan Soto grounds out, second baseman Ryan McM...",608566,665742,FF,4-Seam Fastball,NaN,ball,NaN,9.0,L,Juan Soto,內角低好球
359995,2023,2023-03-30,"Márquez, Germán",Juan Soto strikes out swinging.,608566,665742,FF,4-Seam Fastball,strikeout,swinging_strike,NaN,11.0,L,Juan Soto,外角高壞球


In [60]:
R = ['內角高好球', '中間高好球','外角高好球','內角中間好球','紅中','外角中間好球','內角低好球', '中間低好球','外角低好球','X','內角高壞球','外角高壞球','內角低壞球','外角低壞球']
L = ['外角高好球','中間高好球','內角高好球','外角中間好球','紅中','內角中間好球','外角低好球','中間低好球','內角低好球','X','外角高壞球','內角高壞球','外角低壞球','內角低壞球']

## 計算對該球種打擊率

In [61]:
BA_list = []
for zone in range(14):
    BA_list.append([zone+1])
    for i in range(1):
        player_all_data = batter_data[(batter_data['zone'] == zone+1)]     
        total_hits = player_all_data['events'].str.contains('^(?:single|double|triple|home_run)$').sum()
        total_at_bats = player_all_data['events'].str.contains('^(?:single|double|triple|home_run|field_out|strikeout|force_out|grounded_into_double_play|field_error|double_play|fielders_choice|fielders_choice_out|strikeout_double_play|other_out)$').sum()
        if total_at_bats == 0:
            total_hits = 9999
            total_at_bats = 0.00001
            batting_average = total_hits / total_at_bats
            BA_list[zone].append(batting_average)
        else:   
            batting_average = total_hits / total_at_bats
            BA_list[zone].append(batting_average)
sorted_BA = sorted(BA_list, key=lambda x: x[1])
        
sorted_BA    

[[6, 0.0],
 [12, 0.0],
 [13, 0.0],
 [2, 0.125],
 [3, 0.2],
 [5, 0.2],
 [11, 0.25],
 [7, 0.2857142857142857],
 [4, 0.5],
 [1, 0.6],
 [8, 0.6],
 [9, 1.0],
 [10, 999899999.9999999],
 [14, 999899999.9999999]]

In [62]:
BA_result = []
if batter_data['stand'].iloc[0] == 'R':  
    for item in sorted_BA:
        zone = item[0]
        batting_average = item[1]
        if zone == 1:
            BA_result.append([zone, batting_average, R[0]])
        elif zone == 2:
            BA_result.append([zone, batting_average, R[1]])
        elif zone == 3:
            BA_result.append([zone, batting_average, R[2]])
        elif zone == 4:
            BA_result.append([zone, batting_average, R[3]])
        elif zone == 5:
            BA_result.append([zone, batting_average, R[4]])
        elif zone == 6:
            BA_result.append([zone, batting_average, R[5]])
        elif zone == 7:
            BA_result.append([zone, batting_average, R[6]])
        elif zone == 8:
            BA_result.append([zone, batting_average, R[7]])
        elif zone == 9:
            BA_result.append([zone, batting_average, R[8]])
        elif zone == 10:
            BA_result.append([zone, batting_average, R[9]])
        elif zone == 11:
            BA_result.append([zone, batting_average, R[10]])
        elif zone == 12:
            BA_result.append([zone, batting_average, R[11]])
        elif zone == 13:
            BA_result.append([zone, batting_average, R[12]]) 
        else:
            BA_result.append([zone, batting_average, R[13]])
else:  # stand == 'L'
    for item in sorted_BA:
        zone = item[0]
        batting_average = item[1]
        if zone == 1:
            BA_result.append([zone, batting_average, L[0]])
        elif zone == 2:
            BA_result.append([zone, batting_average, L[1]])
        elif zone == 3:
            BA_result.append([zone, batting_average, L[2]])
        elif zone == 4:
            BA_result.append([zone, batting_average, L[3]])
        elif zone == 5:
            BA_result.append([zone, batting_average, L[4]])
        elif zone == 6:
            BA_result.append([zone, batting_average, L[5]])
        elif zone == 7:
            BA_result.append([zone, batting_average, L[6]])
        elif zone == 8:
            BA_result.append([zone, batting_average, L[7]])
        elif zone == 9:
            BA_result.append([zone, batting_average, L[8]])
        elif zone == 10:
            BA_result.append([zone, batting_average, L[9]])
        elif zone == 11:
            BA_result.append([zone, batting_average, L[10]])
        elif zone == 12:
            BA_result.append([zone, batting_average, L[11]])
        elif zone == 13:
            BA_result.append([zone, batting_average, L[12]])
        else:
            BA_result.append([zone, batting_average, L[13]])

BA_result


[[6, 0.0, '內角中間好球'],
 [12, 0.0, '內角高壞球'],
 [13, 0.0, '外角低壞球'],
 [2, 0.125, '中間高好球'],
 [3, 0.2, '內角高好球'],
 [5, 0.2, '紅中'],
 [11, 0.25, '外角高壞球'],
 [7, 0.2857142857142857, '外角低好球'],
 [4, 0.5, '外角中間好球'],
 [1, 0.6, '外角高好球'],
 [8, 0.6, '中間低好球'],
 [9, 1.0, '內角低好球'],
 [10, 999899999.9999999, 'X'],
 [14, 999899999.9999999, '內角低壞球']]

In [63]:
#顯示打擊率最低的位置(可能有些位置為相同值 一併列出)
smallest = BA_result[0][1]
BA_final_result = []
for item in BA_result:
    if item[1] != smallest:
        break
    BA_final_result.append(item[2])
    
print(pitch_type,'應投在',BA_final_result,'較佳,其打擊率最低')    

FF 應投在 ['內角中間好球', '內角高壞球', '外角低壞球'] 較佳,其打擊率最低


In [64]:
#顯示打擊率前三低的位置(可能有些位置為相同值 一併列出)
three_BA_final_result = []
unique_BA_results = set()

for item in BA_result:
    BA = item[1]
    three_BA_final_result.append(item[2])
    unique_BA_results.add(BA)
    
    if len(unique_BA_results) == 3:
        break

print(pitch_type, '投在', three_BA_final_result, '打擊率較低')

FF 投在 ['內角中間好球', '內角高壞球', '外角低壞球', '中間高好球', '內角高好球'] 打擊率較低


## 計算對該球種揮空率(whiff%)

In [65]:
whiff_list = []
for zone in range(14):
    whiff_list.append([zone+1])
    for i in range(1):
        player_all_data = batter_data[(batter_data['zone'] == zone+1)]
        total_swing_and_miss = player_all_data['description'].str.contains('^(?:swinging_strike|blocked_ball|foul_tip|swinging_strike_blocked|missed_bunt|bunt_foul_tip)$').sum()
        total_ball_swing = player_all_data['description'].str.contains('^(?:foul|swinging_strike|blocked_ball|foul_tip|swinging_strike_blocked|foul_bunt|missed_bunt|bunt_foul_tip)$').sum()
        if total_ball_swing == 0:
            total_swing_and_miss = -0.00001
            total_ball_swing = 1
            whiff_percent = total_swing_and_miss/total_ball_swing
            whiff_list[zone].append(whiff_percent)
        else:   
            whiff_percent = total_swing_and_miss/total_ball_swing
            whiff_list[zone].append(whiff_percent)
                       
sorted_whiff = sorted(whiff_list, key=lambda x: x[1],reverse=True)
           
sorted_whiff

[[9, 1.0],
 [14, 1.0],
 [7, 0.6666666666666666],
 [11, 0.5714285714285714],
 [13, 0.5],
 [1, 0.2857142857142857],
 [2, 0.25],
 [12, 0.25],
 [3, 0.2],
 [4, 0.0],
 [5, 0.0],
 [6, 0.0],
 [8, 0.0],
 [10, -1e-05]]

In [66]:
whiff_result = []
if batter_data['stand'].iloc[0] == 'R':    
    for item in sorted_whiff:
        zone = item[0]
        whiff_percent = item[1]
        if zone == 1:
            whiff_result.append([zone, whiff_percent, R[0]])
        elif zone == 2:
            whiff_result.append([zone, whiff_percent, R[1]])
        elif zone == 3:
            whiff_result.append([zone, whiff_percent, R[2]])
        elif zone == 4:
            whiff_result.append([zone, whiff_percent, R[3]])
        elif zone == 5:
            whiff_result.append([zone, whiff_percent, R[4]])
        elif zone == 6:
            whiff_result.append([zone, whiff_percent, R[5]])
        elif zone == 7:
            whiff_result.append([zone, whiff_percent, R[6]])
        elif zone == 8:
            whiff_result.append([zone, whiff_percent, R[7]])
        elif zone == 9:
            whiff_result.append([zone, whiff_percent, R[8]])
        elif zone == 10:
            whiff_result.append([zone, whiff_percent, R[9]])
        elif zone == 11:
            whiff_result.append([zone, whiff_percent, R[10]])
        elif zone == 12:
            whiff_result.append([zone, whiff_percent, R[11]])
        elif zone == 13:
            whiff_result.append([zone, whiff_percent, R[12]])            
        else:
            whiff_result.append([zone, whiff_percent, R[13]])
else:  # stand == 'L'
    for item in sorted_whiff:
        zone = item[0]
        whiff_percent = item[1]
        if zone == 1:
            whiff_result.append([zone, whiff_percent, L[0]])
        elif zone == 2:
            whiff_result.append([zone, whiff_percent, L[1]])
        elif zone == 3:
            whiff_result.append([zone, whiff_percent, L[2]])
        elif zone == 4:
            whiff_result.append([zone, whiff_percent, L[3]])
        elif zone == 5:
            whiff_result.append([zone, whiff_percent, L[4]])
        elif zone == 6:
            whiff_result.append([zone, whiff_percent, L[5]])
        elif zone == 7:
            whiff_result.append([zone, whiff_percent, L[6]])
        elif zone == 8:
            whiff_result.append([zone, whiff_percent, L[7]])
        elif zone == 9:
            whiff_result.append([zone, whiff_percent, L[8]])
        elif zone == 10:
            whiff_result.append([zone, whiff_percent, L[9]])
        elif zone == 11:
            whiff_result.append([zone, whiff_percent, L[10]])
        elif zone == 12:
            whiff_result.append([zone, whiff_percent, L[11]])
        elif zone == 13:
            whiff_result.append([zone, whiff_percent, L[12]])
        else:
            whiff_result.append([zone, whiff_percent, L[13]])

whiff_result

[[9, 1.0, '內角低好球'],
 [14, 1.0, '內角低壞球'],
 [7, 0.6666666666666666, '外角低好球'],
 [11, 0.5714285714285714, '外角高壞球'],
 [13, 0.5, '外角低壞球'],
 [1, 0.2857142857142857, '外角高好球'],
 [2, 0.25, '中間高好球'],
 [12, 0.25, '內角高壞球'],
 [3, 0.2, '內角高好球'],
 [4, 0.0, '外角中間好球'],
 [5, 0.0, '紅中'],
 [6, 0.0, '內角中間好球'],
 [8, 0.0, '中間低好球'],
 [10, -1e-05, 'X']]

In [67]:
#揮空率最高的位置(可能有些位置為相同值 一併列出)
biggest_whiff = whiff_result[0][1]
whiff_final_result = []
for item in whiff_result:
    if item[1] != biggest_whiff:
        break
    whiff_final_result.append(item[2])
    
print(pitch_type,'投在',whiff_final_result,'揮空率最高') 

FF 投在 ['內角低好球', '內角低壞球'] 揮空率最高


In [68]:
#揮空率前三低的位置(可能有些位置為相同值 一併列出)
three_whiff_final_result = []
unique_whiff_results = set()

for item in whiff_result:
    whiff = item[1]
    three_whiff_final_result.append(item[2])
    unique_whiff_results.add(whiff)
    
    if len(unique_whiff_results) == 3:
        break

print(pitch_type, '投在', three_whiff_final_result, '揮空率較高')

FF 投在 ['內角低好球', '內角低壞球', '外角低好球', '外角高壞球'] 揮空率較高


## 計算對該球種出棒率(swing%)

In [69]:
swing_list = []
for zone in range(14):
    swing_list.append([zone+1])
    for i in range(1):
        player_all_data = batter_data[(batter_data['zone'] == zone+1)]
        total_swing = player_all_data['description'].str.contains('^(?:foul|swinging_strike|hit_into_play|foul_tip|swinging_strike_blocked|missed_bunt|bunt_foul_tip)$').sum()
        total_ball_watch = player_all_data['description'].str.contains('^(?:ball|foul|hit_into_play|called_strike|blocked_ball|swinging_strike|foul_tip|swinging_strike_blocked|hit_by_pitch|foul_bunt|missed_bunt|pitchout|bunt_foul_tip)$').sum()
        if total_ball_watch == 0:
            total_swing = -0.00001
            total_ball_watch = 1
            swing_percent = total_swing / total_ball_watch
            swing_list[zone].append(swing_percent)
        else:   
            swing_percent = total_swing / total_ball_watch
            swing_list[zone].append(swing_percent)

sorted_swing = sorted(swing_list, key=lambda x: x[1],reverse=True)

sorted_swing


[[2, 0.8780487804878049],
 [3, 0.8333333333333334],
 [5, 0.631578947368421],
 [1, 0.6206896551724138],
 [6, 0.4444444444444444],
 [4, 0.38461538461538464],
 [7, 0.375],
 [8, 0.3684210526315789],
 [11, 0.3333333333333333],
 [12, 0.18518518518518517],
 [9, 0.16666666666666666],
 [13, 0.06060606060606061],
 [14, 0.0],
 [10, -1e-05]]

In [70]:
swing_result = []
if batter_data['stand'].iloc[0] == 'R':    
    for item in sorted_swing:
        zone = item[0]
        swing_percent = item[1]
        if zone == 1:
            swing_result.append([zone, swing_percent, R[0]])
        elif zone == 2:
            swing_result.append([zone, swing_percent, R[1]])
        elif zone == 3:
            swing_result.append([zone, swing_percent, R[2]])
        elif zone == 4:
            swing_result.append([zone, swing_percent, R[3]])
        elif zone == 5:
            swing_result.append([zone, swing_percent, R[4]])
        elif zone == 6:
            swing_result.append([zone, swing_percent, R[5]])
        elif zone == 7:
            swing_result.append([zone, swing_percent, R[6]])
        elif zone == 8:
            swing_result.append([zone, swing_percent, R[7]])
        elif zone == 9:
            whiff_result.append([zone, swing_percent, R[8]])
        elif zone == 10:
            swing_result.append([zone, swing_percent, R[9]])
        elif zone == 11:
            swing_result.append([zone, swing_percent, R[10]])
        elif zone == 12:
            swing_result.append([zone, swing_percent, R[11]])
        elif zone == 13:
            swing_result.append([zone, swing_percent, R[12]])
        else:
            swing_result.append([zone, swing_percent, R[13]])
else:  # stand == 'L'
    for item in sorted_swing:
        zone = item[0]
        swing_percent = item[1]
        if zone == 1:
            swing_result.append([zone, swing_percent, L[0]])
        elif zone == 2:
            swing_result.append([zone, swing_percent, L[1]])
        elif zone == 3:
            swing_result.append([zone, swing_percent, L[2]])
        elif zone == 4:
            swing_result.append([zone, swing_percent, L[3]])
        elif zone == 5:
            swing_result.append([zone, swing_percent, L[4]])
        elif zone == 6:
            swing_result.append([zone, swing_percent, L[5]])
        elif zone == 7:
            swing_result.append([zone, swing_percent, L[6]])
        elif zone == 8:
            swing_result.append([zone, swing_percent, L[7]])
        elif zone == 9:
            swing_result.append([zone, swing_percent, L[8]])
        elif zone == 10:
            swing_result.append([zone, swing_percent, L[9]])
        elif zone == 11:
            swing_result.append([zone, swing_percent, L[10]])
        elif zone == 12:
            swing_result.append([zone, swing_percent, L[11]])
        elif zone == 13:
            swing_result.append([zone, swing_percent, L[12]])
        else:
            swing_result.append([zone, swing_percent, L[11]])

swing_result

[[2, 0.8780487804878049, '中間高好球'],
 [3, 0.8333333333333334, '內角高好球'],
 [5, 0.631578947368421, '紅中'],
 [1, 0.6206896551724138, '外角高好球'],
 [6, 0.4444444444444444, '內角中間好球'],
 [4, 0.38461538461538464, '外角中間好球'],
 [7, 0.375, '外角低好球'],
 [8, 0.3684210526315789, '中間低好球'],
 [11, 0.3333333333333333, '外角高壞球'],
 [12, 0.18518518518518517, '內角高壞球'],
 [9, 0.16666666666666666, '內角低好球'],
 [13, 0.06060606060606061, '外角低壞球'],
 [14, 0.0, '內角高壞球'],
 [10, -1e-05, 'X']]

In [71]:
#出棒率最高的位置(可能有些位置為相同值 一併列出)
biggest_swing = swing_result[0][1]
swing_final_result = []
for item in swing_result:
    if item[1] != biggest_swing:
        break
    swing_final_result.append(item[2])
       
print(pitch_type,'投在',swing_final_result,'出棒率最高') 

FF 投在 ['中間高好球'] 出棒率最高


In [72]:
#出棒率前三低的位置(可能有些位置為相同值 一併列出)
three_swing_final_result = []
unique_swing_results = set()

for item in swing_result:
    swing = item[1]
    three_swing_final_result.append(item[2])
    unique_swing_results.add(swing)
    
    if len(unique_swing_results) == 3:
        break

print(pitch_type, '投在', three_swing_final_result, '出棒率最高')

FF 投在 ['中間高好球', '內角高好球', '紅中'] 出棒率最高


## 最終結論整理

In [39]:
#在 statcast_class 資料中判定球員所屬的分類
batter_class = statcast_class.loc[statcast_class['name'] == batter_name, 'class_pred'].iloc[0]

In [40]:
batter_class

'D'

In [41]:
# 印出結果
if batter_class == "A":
    print("球員名稱:", batter_name)
    print("球員分類:", batter_class)
    print(pitch_type, '投在', BA_final_result, '打擊率較低') 
elif batter_class == "B":
    print("球員名稱:", batter_name)
    print("球員分類:", batter_class)
    print(pitch_type, '投在', BA_final_result, '打擊率較低')
    print(pitch_type,'投在',whiff_final_result,'揮空率最高')     
elif batter_class == "C":
    print("球員名稱:", batter_name)
    print("球員分類:", batter_class)
    print(pitch_type, '投在', BA_final_result, '打擊率較低')
    print(pitch_type, '投在', swing_final_result, '出棒率最高')
elif batter_class == "D":
    print("球員名稱:", batter_name)
    print("球員分類:", batter_class)
    print(pitch_type, '投在', BA_final_result, '打擊率較低')
    print(pitch_type, '投在', swing_final_result, '出棒率最高')
else:
    print("球員名稱:", batter_name)
    print("球員分類:", batter_class)
    print(pitch_type, '投在', BA_final_result, '打擊率較低')


球員名稱: Alex Verdugo
球員分類: D
FF 投在 ['外角高好球', '外角低好球', '外角低壞球'] 打擊率較低
FF 投在 ['內角高好球'] 出棒率最高


## 計算特定投手 各球種比例

In [42]:
pitcher_data = statcast[statcast['player_name'] == pitcher_name ]

# 計算每個pitch_type的出現次數
pitch_type_counts = pitcher_data['pitch_type'].value_counts()

# 計算每個pitch_type的比例
pitch_type_proportions = pitch_type_counts / pitch_type_counts.sum()

print(pitch_type_proportions)

FF    0.563950
SL    0.219512
KC    0.119572
CH    0.074955
FC    0.022011
Name: pitch_type, dtype: float64


In [43]:
pitcher_data = statcast[statcast['player_name'] == pitcher_name]

# 計算每個pitch_name的出現次數
pitch_type_counts = pitcher_data['pitch_name'].value_counts()

# 計算每個pitch_name的比例
pitch_type_proportions = pitch_type_counts / pitch_type_counts.sum()

print(pitch_type_proportions)

4-Seam Fastball    0.563950
Slider             0.219512
Knuckle Curve      0.119572
Changeup           0.074955
Cutter             0.022011
Name: pitch_name, dtype: float64


## 計算特定投手 各球種各位置使用比例

In [44]:
# 根據'zone'和'pitch_name'進行分組，計算每個分組的計數
grouped_data = pitcher_data.groupby(['zone', 'pitch_name']).size().reset_index(name='count')

# 計算每個'zone'中的總計數
zone_counts = grouped_data.groupby('zone')['count'].sum()

# 遍歷所有的'zone'和'pitch_name'組合，計算使用率並存儲在一個字典中
pitch_usage = {}
for index, row in grouped_data.iterrows():
    zone = row['zone']
    pitch_name = row['pitch_name']
    count = row['count']
    usage = count / zone_counts[zone]
    if zone not in pitch_usage:
        pitch_usage[zone] = {}
    pitch_usage[zone][pitch_name] = usage

pitch_usage

{1.0: {'4-Seam Fastball': 0.7934782608695652,
  'Changeup': 0.043478260869565216,
  'Cutter': 0.021739130434782608,
  'Knuckle Curve': 0.06521739130434782,
  'Slider': 0.07608695652173914},
 2.0: {'4-Seam Fastball': 0.8118811881188119,
  'Changeup': 0.0297029702970297,
  'Cutter': 0.009900990099009901,
  'Knuckle Curve': 0.06930693069306931,
  'Slider': 0.07920792079207921},
 3.0: {'4-Seam Fastball': 0.8709677419354839,
  'Cutter': 0.016129032258064516,
  'Knuckle Curve': 0.04838709677419355,
  'Slider': 0.06451612903225806},
 4.0: {'4-Seam Fastball': 0.6301369863013698,
  'Changeup': 0.1506849315068493,
  'Knuckle Curve': 0.0958904109589041,
  'Slider': 0.1232876712328767},
 5.0: {'4-Seam Fastball': 0.6230769230769231,
  'Changeup': 0.03076923076923077,
  'Cutter': 0.015384615384615385,
  'Knuckle Curve': 0.1076923076923077,
  'Slider': 0.2230769230769231},
 6.0: {'4-Seam Fastball': 0.648,
  'Changeup': 0.008,
  'Cutter': 0.04,
  'Knuckle Curve': 0.072,
  'Slider': 0.232},
 7.0: {'4-S

## 計算特定投手對特定打者 各球種各位置使用比例

In [45]:
# 篩選出屬於投手面對特定打者的資料
pither_face_batter_data = statcast[(statcast['player_name'] == pitcher_name) & (statcast['batter_name'] == batter_name)]

# 根據'zone'和'pitch_name'進行分組，計算每個分組的計數
grouped_data = pither_face_batter_data.groupby(['zone', 'pitch_name']).size().reset_index(name='count')

# 計算每個'zone'中的總計數
zone_counts = grouped_data.groupby('zone')['count'].sum()

# 遍歷所有的'zone'和'pitch_name'組合，計算使用率並存儲在一個字典中
pitch_usage = {}
for index, row in grouped_data.iterrows():
    zone = row['zone']
    pitch_name = row['pitch_name']
    count = row['count']
    usage_percent = count / zone_counts[zone]
    if zone not in pitch_usage:
        pitch_usage[zone] = {}
    pitch_usage[zone][pitch_name] = usage

pitch_usage


{4.0: {'Changeup': 0.514745308310992},
 5.0: {'4-Seam Fastball': 0.514745308310992},
 6.0: {'4-Seam Fastball': 0.514745308310992},
 7.0: {'4-Seam Fastball': 0.514745308310992},
 9.0: {'Cutter': 0.514745308310992},
 11.0: {'4-Seam Fastball': 0.514745308310992},
 12.0: {'4-Seam Fastball': 0.514745308310992},
 13.0: {'4-Seam Fastball': 0.514745308310992, 'Changeup': 0.514745308310992},
 14.0: {'Slider': 0.514745308310992}}

## 計算特定投手對特定打者 各球種各位置使用個數

In [46]:
import pandas as pd

# 篩選出屬於"Germán, Domingo"面對特定打者的資料
pither_face_batter_data = statcast[(statcast['player_name'] == pitcher_name) & (statcast['batter_name'] == batter_name)]

# 根據"zone"和"pitch_name"進行分組，計算每個分組的次數
grouped_data = pither_face_batter_data.groupby(['zone', 'pitch_name']).size().reset_index(name='count')

grouped_data

,zone,pitch_name,count
0,4.0,Changeup,1
1,5.0,4-Seam Fastball,1
2,6.0,4-Seam Fastball,1
3,7.0,4-Seam Fastball,2
4,9.0,Cutter,1
5,11.0,4-Seam Fastball,1
6,12.0,4-Seam Fastball,1
7,13.0,4-Seam Fastball,2
8,13.0,Changeup,2
9,14.0,Slider,2


## 驗算區域

In [47]:
#player_name ="Mike Trout"
player_data2 = statcast[(statcast['batter_name'] == batter_name)]

In [48]:
player_data2

,game_year,game_date,player_name,des,pitcher,batter,pitch_type,pitch_name,events,description,bb_type,zone,stand,batter_name
2215,2023,2023-06-30,"Thornton, Trent",Alex Verdugo singles on a ground ball to third...,663423,657077,FF,4-Seam Fastball,single,hit_into_play,ground_ball,11.0,L,Alex Verdugo
2216,2023,2023-06-30,"Thornton, Trent",Alex Verdugo singles on a ground ball to third...,663423,657077,FF,4-Seam Fastball,NaN,ball,NaN,11.0,L,Alex Verdugo
2217,2023,2023-06-30,"Thornton, Trent",Alex Verdugo singles on a ground ball to third...,663423,657077,CU,Curveball,NaN,foul,NaN,9.0,L,Alex Verdugo
2218,2023,2023-06-30,"Thornton, Trent",Alex Verdugo singles on a ground ball to third...,663423,657077,CU,Curveball,NaN,ball,NaN,14.0,L,Alex Verdugo
2219,2023,2023-06-30,"Thornton, Trent",Alex Verdugo singles on a ground ball to third...,663423,657077,FF,4-Seam Fastball,NaN,called_strike,NaN,13.0,L,Alex Verdugo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358749,2023,2023-03-30,"Gibson, Kyle",Alex Verdugo grounds out to first baseman Ryan...,502043,657077,CH,Changeup,field_out,hit_into_play,ground_ball,8.0,L,Alex Verdugo
358750,2023,2023-03-30,"Gibson, Kyle",Alex Verdugo grounds out to first baseman Ryan...,502043,657077,FF,4-Seam Fastball,NaN,ball,NaN,11.0,L,Alex Verdugo
358751,2023,2023-03-30,"Gibson, Kyle",Alex Verdugo grounds out to first baseman Ryan...,502043,657077,CU,Curveball,NaN,called_strike,NaN,6.0,L,Alex Verdugo
358825,2023,2023-03-30,"Gibson, Kyle",Alex Verdugo triples (1) on a sharp fly ball t...,502043,657077,FF,4-Seam Fastball,triple,hit_into_play,fly_ball,5.0,L,Alex Verdugo


In [49]:
total_swing = player_data2['description'].str.contains('^(?:foul|swinging_strike|hit_into_play|foul_tip|swinging_strike_blocked|missed_bunt|bunt_foul_tip)$').sum()
total_ball_watch = player_data2['description'].str.contains('^(?:ball|foul|hit_into_play|called_strike|blocked_ball|swinging_strike|foul_tip|swinging_strike_blocked|hit_by_pitch|foul_bunt|missed_bunt|pitchout|bunt_foul_tip)$').sum()
swing_percent = total_swing / total_ball_watch
swing_percent

0.396327467482785

In [50]:
total_swing = player_data2['description'].str.contains('^(?:foul|swinging_strike|hit_into_play|foul_tip|swinging_strike_blocked|missed_bunt|bunt_foul_tip)$').sum()
total_ball_watch = len(player_data2)
swing_percent = total_swing / total_ball_watch
swing_percent

0.396327467482785

In [51]:
total_swing_and_miss = player_data2['description'].str.contains('^(?:swinging_strike|foul_tip|swinging_strike_blocked|missed_bunt|bunt_foul_tip)$').sum()
total_ball_swing = player_data2['description'].str.contains('^(?:foul|swinging_strike|hit_into_play|foul_tip|swinging_strike_blocked|missed_bunt|bunt_foul_tip)$').sum()
whiff_percent = total_swing_and_miss/total_ball_swing
whiff_percent

0.14092664092664092

In [52]:
statcast.pitch_name.value_counts(dropna=False)

4-Seam Fastball    117052
Slider              65247
Sinker              54714
Changeup            39146
Cutter              28239
Curveball           25994
Sweeper             15269
Split-Finger         7957
Knuckle Curve        6742
Slurve               1246
Other                 574
Forkball              419
Eephus                243
NaN                    76
Screwball              52
Slow Curve             29
Pitch Out              20
Knuckleball            18
Name: pitch_name, dtype: int64